In [1]:
import os

In [2]:
from modules.pipeline import *

modules.report successfull loaded: 100%|██████████| 6/6 [00:03<00:00,  1.68it/s]


In [3]:
from unidecode import unidecode

In [4]:
import re

In [5]:
from copy import deepcopy

In [6]:
def get_filenames(root_dir, func, verbose=False):
    data_filenames = []
    # Walk through the directories and files
    for dirpath, dirnames, filenames in os.walk(root_dir):
        # dirpath: current directory path
        # dirnames: list of directories in the current directory
        # filenames: list of files in the current directory

        # Print the current directory
        print('Directory:', dirpath)  if verbose else None
        # Print all the subdirectories
        if verbose:
            for dirname in dirnames:
                print('Subdirectory:', os.path.join(dirpath, dirname))

        # Print all the files
        for filename in filenames:
            if func(filename) and not ('x_' in filename or 'y_' in filename or 'metric' in filename):
                print('File:', os.path.join(dirpath, filename)) if verbose else None
                data_filenames.append(os.path.join(dirpath, filename))

        # Print an empty line to separate directories
        print()  if verbose else None
    return data_filenames

In [7]:
MLN_F = lambda x: (('classic_mln_' in x) and not('classic_mln_-' in x)) # find metric of model where mln were added
MLN__F = lambda x: (('classic_mln_-' in x)) # where mln attribut were removed first
MLN_C_F = lambda x: (('classic_-' in x)) # where mln attribut were removed first
MLN_C= lambda x: (('classic_' in x)) # where mln attribut were removed first

INTER_F = lambda x: (not('_max_' in x) and ('inter' in x))
INTRA_F = lambda x: (not('_max_' in x) and ('intra' in x))
COMBINE_F = lambda x: (not('_max_' in x) and ('combine' in x))
ULTRA_F = lambda x: (not('_max_' in x) and ('ultra' in x))
INTER_MAX_F = lambda x: (('_max_' in x) and ('inter' in x))
INTRA_MAX_F = lambda x: (('_max_' in x) and ('intra' in x))
COMBINE_MAX_F = lambda x: (('_max_' in x) and ('combine' in x))
ULTRA_MAX_F = lambda x: (('_max_' in x) and ('ultra' in x))
DEGREE_F = lambda x: (('degree' in x))

In [8]:
def metrics_analyzer_statistics_tab_f_1(cols=None, outputs_path=None, cwd=None, data_folder=None, classic_metrics=None, models_name=None, layers=[1], approach=None):
    """ build relevance results about the datasset
    
    Args:
        - cols: list of qualitative variable in the dataset
        - outputs_path: the path where the experimental results are located
        - 
    
    Returns:
        A dedicated folder with those relevante reports and charts
    
    """
    day = time.strftime("%Y_%m_%d_%H")
    if cols != None or classic_metrics != None: # check if cols and classics metrics are filled
        ## analyse of k layer

        # find out all best metric details
        head_lambda = lambda x: f'<tr><td colspan="2" rowspan="2">{x}</td><td colspan="2">Accuracy</td><td colspan="2">Precision</td><td colspan="2">Recall</td><td colspan="2">F1-score</td></tr><tr>{"<td>G</td><td>P</td>"*4}</tr>'
        tab1_head = head_lambda(data_folder)
        tab1_body = ""
        metrics = {
            'accuracy': {
                'classic_mln':{
                    'P':[],
                    'G':[]
                },
                'classic_mln_-_mlna':{
                    'P':[],
                    'G':[]
                }
            },
            'precision': {
                'classic_mln':{
                    'P':[],
                    'G':[]
                },
                'classic_mln_-_mlna':{
                    'P':[],
                    'G':[]
                }
            },
            'recall': {
                'classic_mln':{
                    'P':[],
                    'G':[]
                },
                'classic_mln_-_mlna':{
                    'P':[],
                    'G':[]
                }
            },
            'f1-score': {
                'classic_mln':{
                    'P':[],
                    'G':[]
                },
                'classic_mln_-_mlna':{
                    'P':[],
                    'G':[]
                }
            }
        }

        totalImpact = {
            'accuracy': {
                    'P':[],
                    'G':[]
                },
            'precision': {
                    'P':[],
                    'G':[]
                },
            'recall': {
                    'P':[],
                    'G':[]
                },
            'f1-score': {
                    'P':[],
                    'G':[]
                }
        }
        
        dictio = {
                    'classic_-_mlna': 'Classic - Att',
                    'classic_mln': 'Classic + MLN',
                    'classic_mln_-_mlna': 'Classic + MLN - Att',
                    'classic': 'Classic'
                }
        
        
        style = '<style> table, th, td {border: 1px solid black;border-collapse: collapse;} .wrap-text {word-wrap: break-word;} .wrap-text {overflow-wrap: break-word;} .limited-column {width: 100px;} .dashed-border {border: 1px dashed black;}.dotted-border {border: 1px dotted black;} td {text-align: center;} caption {margin:0; margin-bottom: 2px; text-align: start; border: 1px dashed black;} caption > h2 {text-align: center;}</style>'
        
        caption_content_lambda = lambda x: ''.join([f'<span><strong>{key}</strong>: {value}</span><br>' for key, value in {
            **models_name,
            'MLN k Layer(s)': 'MultiLayer Network with k layer(s)',
            'Att': 'Attributs or modalities of variable(s) used to build MLN',
            'MLN': 'Descriptors extracted from MLN',
            'Classic': f'Learning from classic dataset of {data_folder}',
            'Classic - Att': f'Learning from classic dataset of {data_folder} where Att had been removed',
            'Classic + MLN': f'Learning from classic dataset of {data_folder} where MLN had been added',
            'Classic + MLN - Att': f'Learning from classic dataset of {data_folder} where MLN had been added and Att removed'
            }.items()])
        
        tab1_body_model = {key: deepcopy(metrics) for key in models_name.keys()}

        # fetch layers
        for k in layers:
            # fetch each combinantion of atributs in layers
            for layer_config in get_combinations(range(len(cols)),k): # create subsets of k index of OHE and fetch it
                col_targeted= [f'{cols[i]}' for i in layer_config]
                case_k= '±'.join(col_targeted) if len(layer_config)>1 else col_targeted[0]
                
                ### get files for distincts logic
                match= lambda x: (
                    sum(
                        [
                            re.sub(r'[^\w\s]', '', unidecode(partern)) in re.sub(r'[^\w\s]', '', unidecode(x))
                            for partern in case_k.split("±")
                            ]
                        ) == k if k > 1 else re.sub(r'[^\w\s]', '', unidecode(case_k)) in re.sub(r'[^\w\s]', '', unidecode(x))
                    )
                files = {
                    'global':{
                        'classic': classic_metrics,
                        #'classic_-_mlna':[
                        #    load_data_set_from_url(path=file,sep='\t', encoding='utf-8',index_col=0, na_values=None) 
                        #    for file in get_filenames(
                        #        root_dir=f'{outputs_path}/qualitative/mlna_{k}/data_selection_storage', 
                        #        func=lambda x: ((MLN_C_F(x)) and (match(x))), 
                        #        verbose=False
                         #       )
                         #   ][-1],
                        'classic_mln':[
                            load_data_set_from_url(path=file,sep='\t', encoding='utf-8',index_col=0, na_values=None) 
                            for file in get_filenames(
                                root_dir=f'{outputs_path}/qualitative/mlna_{k}/global/data_selection_storage', 
                                func=lambda x: ((MLN_F(x)) and (match(x))), 
                                verbose=False
                                )
                            ][-1],
                        'classic_mln_-_mlna':[
                            load_data_set_from_url(path=file,sep='\t', encoding='utf-8',index_col=0, na_values=None) 
                            for file in get_filenames(
                                root_dir=f'{outputs_path}/qualitative/mlna_{k}/global/data_selection_storage', 
                                func=lambda x: ((MLN__F(x)) and (match(x))), 
                                verbose=False
                                )
                            ][-1]
                    },
                    "personalized":{
                        'classic': classic_metrics,
                       # 'classic_-_mlna':[
                       #     load_data_set_from_url(path=file,sep='\t', encoding='utf-8',index_col=0, na_values=None) 
                       #     for file in get_filenames(
                       #         root_dir=f'{outputs_path}/qualitative/mlna_{k}/data_selection_storage', 
                       #         func=lambda x: ((MLN_C_F(x)) and (match(x))), 
                       #         verbose=False
                       #         )
                       #     ][-1],
                        'classic_mln':[
                            load_data_set_from_url(path=file,sep='\t', encoding='utf-8',index_col=0, na_values=None) 
                            for file in get_filenames(
                                root_dir=f'{outputs_path}/qualitative/mlna_{k}/personalized/data_selection_storage', 
                                func=lambda x: ((MLN_F(x)) and (match(x))), 
                                verbose=False
                                )
                            ][-1],
                        'classic_mln_-_mlna':[
                            load_data_set_from_url(path=file,sep='\t', encoding='utf-8',index_col=0, na_values=None) 
                            for file in get_filenames(
                                root_dir=f'{outputs_path}/qualitative/mlna_{k}/personalized/data_selection_storage', 
                                func=lambda x: ((MLN__F(x)) and (match(x))), 
                                verbose=False
                                )
                            ][-1]
                    }
                }
                # print(files)
                #outputs[logic] = files
                ### transform and normalize
                models_list = files['personalized']['classic'].index.values.tolist()
                print(models_list)
                metrics = ["accuracy","precision","recall","f1-score"]
                
                # fetch models
                for model in models_list:
                    # fetch evaluation metric
                    for metric in metrics:
                        # fetch approach
                        for i, key in enumerate(approach): 
                            # add metric in the vector
                            tab1_body_model[model][metric][key]['G'].append(round(files['global'][key].loc[model,metric],4))
                            tab1_body_model[model][metric][key]['P'].append(round(files['personalized'][key].loc[model,metric],4))
                            #totalImpact[metric]["P"].append(tab1_body_model[model][metric][key]['P'] >= tab1_body_model[model][metric][key]['G'])
                            #totalImpact[metric]["G"].append(tab1_body_model[model][metric][key]['P'] <= tab1_body_model[model][metric][key]['P'])
        
        # fetch each model
        for model in models_list:
            tab1_body+= f'<tr> <td rowspan="2">{model}</td>'
            # fetch approach
            for i, key in enumerate(files['global'].keys() if approach == None else approach):
                # fetch evaluation metric
                tab1_body+= f'<tr> <td>{dictio[key]}</td>' if i != 0 else f'<td>{dictio[key]}</td>'
                for y, metric in enumerate(metrics):
                    # add metric in the vector
                    totalImpact[metric]["P"].append(max(tab1_body_model[model][metric][key]["G"]) <= max(tab1_body_model[model][metric][key]["P"]))
                    totalImpact[metric]["G"].append(max(tab1_body_model[model][metric][key]["G"]) >= max(tab1_body_model[model][metric][key]["P"]))
        
                    tab1_body+= (f'<td>{"<strong>"*int(max(tab1_body_model[model][metric][key]["G"]) >= max(tab1_body_model[model][metric][key]["P"]))}{max(tab1_body_model[model][metric][key]["G"])}{"</strong>"*int(max(tab1_body_model[model][metric][key]["G"]) >= max(tab1_body_model[model][metric][key]["P"]))}</td>'+
                                f'<td> {"<strong>"*int(max(tab1_body_model[model][metric][key]["G"]) <= max(tab1_body_model[model][metric][key]["P"]))}{max(tab1_body_model[model][metric][key]["P"])}{"</strong>"*int(max(tab1_body_model[model][metric][key]["G"]) <= max(tab1_body_model[model][metric][key]["P"]))}</td>'
                                ) if y != len(metrics)-1 else (f'<td>{"<strong>"*int(max(tab1_body_model[model][metric][key]["G"]) >= max(tab1_body_model[model][metric][key]["P"]))}{max(tab1_body_model[model][metric][key]["G"])}{"</strong>"*int(max(tab1_body_model[model][metric][key]["G"]) >= max(tab1_body_model[model][metric][key]["P"]))}</td>'+
                                f'<td> {"<strong>"*int(max(tab1_body_model[model][metric][key]["G"]) <= max(tab1_body_model[model][metric][key]["P"]))}{max(tab1_body_model[model][metric][key]["P"])}{"</strong>"*int(max(tab1_body_model[model][metric][key]["G"]) <= max(tab1_body_model[model][metric][key]["P"]))}</td></tr>')
        tab1_body+= f'<tr> <td colspan="2">Total</td>'
        for y, metric in enumerate(metrics): 
            tab1_body+= (f'<td>{"<strong>"*int(sum(totalImpact[metric]["G"]) >= sum(totalImpact[metric]["P"]))}{sum(totalImpact[metric]["G"])}{"</strong>"*int(sum(totalImpact[metric]["G"]) >= sum(totalImpact[metric]["P"]))}</td>'+
                            f'<td>{"<strong>"*int(sum(totalImpact[metric]["G"]) <= sum(totalImpact[metric]["P"]))}{sum(totalImpact[metric]["P"])}{"</strong>"*int(sum(totalImpact[metric]["G"]) <= sum(totalImpact[metric]["P"]))}</td>'
                            ) if y != len(metrics)-1 else (f'<td>{"<strong>"*int(sum(totalImpact[metric]["G"]) >= sum(totalImpact[metric]["P"]))}{sum(totalImpact[metric]["G"])}{"</strong>"*int(sum(totalImpact[metric]["G"]) >= sum(totalImpact[metric]["P"]))}</td>'+
                            f'<td>{"<strong>"*int(sum(totalImpact[metric]["G"]) <= sum(totalImpact[metric]["P"]))}{sum(totalImpact[metric]["P"])}{"</strong>"*int(sum(totalImpact[metric]["G"]) <= sum(totalImpact[metric]["P"]))}</td></tr>')
            
        
        caption = f'<caption><h2>Legend</h2>{caption_content_lambda(metric)}</caption>'
        table_html = f'<table style="border: 2px solid black; width: 100% !important; background-color: #FFFFFF; color:#000000;">{caption}{tab1_head}{tab1_body}</table>'
        htm = f'<html><head>{style}<title> Best metrics of personalized and global logic for Classic + MLN et Classic + MLN - Att pour chaque modèle </title></head><body style="background-color: white;">{table_html}</body></html>'
        
        create_domain(f'{cwd}/analyze_{outputs_path.split("/")[-2]}_made_on_{day}H/{data_folder}/plots/fnTab/tab1')
        timestr = time.strftime("%Y_%m_%d_%H_%M_%S")
        filename1 = f'{cwd}/analyze_{outputs_path.split("/")[-2]}_made_on_{day}H/{data_folder}/plots/fnTab/tab1/Statistical comparaison of approachs in {data_folder} on global logic'+'_'+timestr+'.html'
        _file= open(filename1,"w")
        _file.write(htm)
        _file.close()
        

In [9]:
get_qualitative_from_cols = lambda x:(list(set([
    var.split("__")[1] for var in [
        coll 
        for coll in [
            col 
            for col in x 
                if not (
                    ('precision' in col ) 
                    or ('accuracy' in col ) 
                    or ('recall' in col) 
                    or ('f1-score' in col)
                )
            ] 
            if ("__" in coll)
        ]
    ]
)))

In [10]:
def analyzer_launcher(outputs_name=None, analytical_func=None, layers=None, approach=None):
    
    result_folders = [dirnames for _, dirnames, _ in os.walk(f'{os.getcwd()}/{outputs_name}')][0]
    for dataset_name in result_folders:
        print(dataset_name)
        classic_f = [
                        load_data_set_from_url(path=file,sep='\t', encoding='utf-8',index_col=0, na_values=None) 
                        for file in get_filenames(
                            root_dir=f'{os.getcwd()}/{outputs_name}/{dataset_name}/data_selection_storage', 
                            func=MLN_C, 
                            verbose=False
                            )
                        ][-1]
        quali_col = get_qualitative_from_cols(classic_f.columns.to_list())
        models_list = classic_f.index.values.tolist()
        models = model_desc()
        models_name = { key : models[key] for key in models.keys() if key in models_list}
        layers = list(set([1, 2, len(quali_col)]))
        analytical_func(
            cols=quali_col, 
            outputs_path=f'{os.getcwd()}/{outputs_name}/{dataset_name}', 
            cwd=os.getcwd(), 
            data_folder=dataset_name, 
            classic_metrics=classic_f, 
            models_name=models_name,
            layers= layers if layers != None else list(set([1, 2, len(quali_col)])), 
            approach= approach
            )
        

In [11]:
layers = None
approach = ['classic_mln','classic_mln_-_mlna']

In [12]:
analyzer_launcher(outputs_name="outputs_16032024", analytical_func=metrics_analyzer_statistics_tab_f_1, layers=layers, approach=approach)

AER
['xgb', 'dtc', 'lrc', 'rfc', 'sv']
['xgb', 'dtc', 'lrc', 'rfc', 'sv']
['xgb', 'dtc', 'lrc', 'rfc', 'sv']
Directory '/Users/djiemboutientcheuvictornico/Documents/M2_Thesis/M2_thesis/scripting/analyze_outputs_16032024_made_on_2024_03_28_09H/AER/plots/fnTab/tab1' created successfully.
AFB
['xgb', 'dtc', 'lrc', 'rfc', 'sv']
['xgb', 'dtc', 'lrc', 'rfc', 'sv']
['xgb', 'dtc', 'lrc', 'rfc', 'sv']
['xgb', 'dtc', 'lrc', 'rfc', 'sv']
['xgb', 'dtc', 'lrc', 'rfc', 'sv']
['xgb', 'dtc', 'lrc', 'rfc', 'sv']
['xgb', 'dtc', 'lrc', 'rfc', 'sv']
['xgb', 'dtc', 'lrc', 'rfc', 'sv']
['xgb', 'dtc', 'lrc', 'rfc', 'sv']
['xgb', 'dtc', 'lrc', 'rfc', 'sv']
['xgb', 'dtc', 'lrc', 'rfc', 'sv']
Directory '/Users/djiemboutientcheuvictornico/Documents/M2_Thesis/M2_thesis/scripting/analyze_outputs_16032024_made_on_2024_03_28_09H/AFB/plots/fnTab/tab1' created successfully.
CREDIT_RISK_DATASET
['xgb', 'dtc', 'lrc', 'rfc', 'sv']
['xgb', 'dtc', 'lrc', 'rfc', 'sv']
['xgb', 'dtc', 'lrc', 'rfc', 'sv']
['xgb', 'dtc', 'lrc',

In [13]:
def metrics_analyzer_statistics_tab_f_2(cols=None, outputs_path=None, cwd=None, data_folder=None, classic_metrics=None, models_name=None, layers=[1], approach=None):
    """ build relevance results about the datasset
    
    Args:
        - cols: list of qualitative variable in the dataset
        - outputs_path: the path where the experimental results are located
        - 
    
    Returns:
        A dedicated folder with those relevante reports and charts
    
    """
    day = time.strftime("%Y_%m_%d_%H")
    if cols != None or classic_metrics != None: # check if cols and classics metrics are filled
        ## analyse of k layer

        # find out all best metric details
        mlnL = {f'MLN {key}': {
                    'P':[],
                    'G':[]
                } for key in layers}
        totalImpact = {
            'accuracy': {f'MLN {key}': [] for key in layers},
            'precision': {f'MLN {key}': [] for key in layers},
            'recall': {f'MLN {key}': [] for key in layers},
            'f1-score': {f'MLN {key}': [] for key in layers}
        }
        head_lambda = lambda x: f'<tr><td colspan="3" rowspan="2">{x}</td><td colspan="{len(mlnL)}">Accuracy</td><td colspan="{len(mlnL)}">Precision</td><td colspan="{len(mlnL)}">Recall</td><td colspan="{len(mlnL)}">F1-score</td></tr><tr>{"".join(["<td>"+key+"</td>" for key in mlnL.keys()]) *4}</tr>'
        tab1_head = head_lambda(data_folder)
        tab1_body = ""
        
        metrics = {
            'accuracy': {
                'classic_mln':deepcopy(mlnL),
                'classic_mln_-_mlna':deepcopy(mlnL)
            },
            'precision': {
                'classic_mln':deepcopy(mlnL),
                'classic_mln_-_mlna':deepcopy(mlnL)
            },
            'recall': {
                'classic_mln':deepcopy(mlnL),
                'classic_mln_-_mlna':deepcopy(mlnL)
            },
            'f1-score': {
                'classic_mln':deepcopy(mlnL),
                'classic_mln_-_mlna':deepcopy(mlnL)
            }
        }
        
            
        dictio = {
            'classic_-_mlna': 'Classic - Att',
            'classic_mln': 'Classic + MLN',
            'classic_mln_-_mlna': 'Classic + MLN - Att',
            'classic': 'Classic'
        }
        
        style = '<style> table, th, td {border: 1px solid black;border-collapse: collapse;} .wrap-text {word-wrap: break-word;} .wrap-text {overflow-wrap: break-word;} .limited-column {width: 100px;} .dashed-border {border: 1px dashed black;}.dotted-border {border: 1px dotted black;} td {text-align: center;} caption {margin:0; margin-bottom: 2px; text-align: start; border: 1px dashed black;} caption > h2 {text-align: center;}</style>'
        
        caption_content_lambda = lambda x: ''.join([f'<span><strong>{key}</strong>: {value}</span><br>' for key, value in {
            **models_name,
            'MLN k Layer(s)': 'MultiLayer Network with k layer(s)',
            'Att': 'Attributs or modalities of variable(s) used to build MLN',
            'MLN': 'Descriptors extracted from MLN',
            'Classic': f'Learning from classic dataset of {data_folder}',
            'Classic - Att': f'Learning from classic dataset of {data_folder} where Att had been removed',
            'Classic + MLN': f'Learning from classic dataset of {data_folder} where MLN had been added',
            'Classic + MLN - Att': f'Learning from classic dataset of {data_folder} where MLN had been added and Att removed'
            }.items()])
        
        tab1_body_model = {key: deepcopy(metrics) for key in models_name.keys()}

        # fetch layers
        for k in layers:
            # fetch each combinantion of atributs in layers
            for layer_config in get_combinations(range(len(cols)),k): # create subsets of k index of OHE and fetch it
                col_targeted= [f'{cols[i]}' for i in layer_config]
                case_k= '±'.join(col_targeted) if len(layer_config)>1 else col_targeted[0]
                
                ### get files for distincts logic
                match= lambda x: (
                    sum(
                        [
                            re.sub(r'[^\w\s]', '', unidecode(partern)) in re.sub(r'[^\w\s]', '', unidecode(x))
                            for partern in case_k.split("±")
                            ]
                        ) == k if k > 1 else re.sub(r'[^\w\s]', '', unidecode(case_k)) in re.sub(r'[^\w\s]', '', unidecode(x))
                    )
                files = {
                    'global':{
                        'classic': classic_metrics,
                        #'classic_-_mlna':[
                        #    load_data_set_from_url(path=file,sep='\t', encoding='utf-8',index_col=0, na_values=None) 
                        #    for file in get_filenames(
                        #        root_dir=f'{outputs_path}/qualitative/mlna_{k}/data_selection_storage', 
                        #        func=lambda x: ((MLN_C_F(x)) and (match(x))), 
                        #        verbose=False
                         #       )
                         #   ][-1],
                        'classic_mln':[
                            load_data_set_from_url(path=file,sep='\t', encoding='utf-8',index_col=0, na_values=None) 
                            for file in get_filenames(
                                root_dir=f'{outputs_path}/qualitative/mlna_{k}/global/data_selection_storage', 
                                func=lambda x: ((MLN_F(x)) and (match(x))), 
                                verbose=False
                                )
                            ][-1],
                        'classic_mln_-_mlna':[
                            load_data_set_from_url(path=file,sep='\t', encoding='utf-8',index_col=0, na_values=None) 
                            for file in get_filenames(
                                root_dir=f'{outputs_path}/qualitative/mlna_{k}/global/data_selection_storage', 
                                func=lambda x: ((MLN__F(x)) and (match(x))), 
                                verbose=False
                                )
                            ][-1]
                    },
                    "personalized":{
                        'classic': classic_metrics,
                       # 'classic_-_mlna':[
                       #     load_data_set_from_url(path=file,sep='\t', encoding='utf-8',index_col=0, na_values=None) 
                       #     for file in get_filenames(
                       #         root_dir=f'{outputs_path}/qualitative/mlna_{k}/data_selection_storage', 
                       #         func=lambda x: ((MLN_C_F(x)) and (match(x))), 
                       #         verbose=False
                       #         )
                       #     ][-1],
                        'classic_mln':[
                            load_data_set_from_url(path=file,sep='\t', encoding='utf-8',index_col=0, na_values=None) 
                            for file in get_filenames(
                                root_dir=f'{outputs_path}/qualitative/mlna_{k}/personalized/data_selection_storage', 
                                func=lambda x: ((MLN_F(x)) and (match(x))), 
                                verbose=False
                                )
                            ][-1],
                        'classic_mln_-_mlna':[
                            load_data_set_from_url(path=file,sep='\t', encoding='utf-8',index_col=0, na_values=None) 
                            for file in get_filenames(
                                root_dir=f'{outputs_path}/qualitative/mlna_{k}/personalized/data_selection_storage', 
                                func=lambda x: ((MLN__F(x)) and (match(x))), 
                                verbose=False
                                )
                            ][-1]
                    }
                }
                # print(files)
                #outputs[logic] = files
                ### transform and normalize
                models_list = files['personalized']['classic'].index.values.tolist()
                print(models_list)
                metrics = ["accuracy","precision","recall","f1-score"]
                
                # fetch models
                for model in models_list:
                    # fetch evaluation metric
                    for metric in metrics:
                        # fetch approach
                        for i, key in enumerate(approach): 
                            # add metric in the vector
                            tab1_body_model[model][metric][key][f'MLN {k}']["G"].append(round(files['global'][key].loc[model,metric],4))
                            tab1_body_model[model][metric][key][f'MLN {k}']["P"].append(round(files['personalized'][key].loc[model,metric],4))
                            
        logics = {"G":"Global","P":"Personalized"}    
        # fetch each model
        for model in models_list:
                tab1_body+= f'<tr> <td rowspan="4">{model}</td>'
                # fetch approach
                for i, key in enumerate(files['global'].keys() if approach == None else approach):
                    # fetch evaluation metric
                    tab1_body+= (f'<tr> <td rowspan="2">{dictio[key]}</td>') if i != 0 else (f'<td rowspan="2">{dictio[key]}</td>')
                    for l,logic in enumerate(logics.keys()):
                        tab1_body+= (f'<tr><td>{logics[logic]}</td>') if l != 0 else (f'<td>{logics[logic]}</td>')
                        for y, metric in enumerate(metrics):
                            #print(f"{y}--")
                            # fetch layers
                            for z, layer in enumerate(mlnL.keys()):
                                #print(f"{z};")
                                # add metric in the vector
                                maxi = [max(tab1_body_model[model][metric][key][lay][logic]) for lay in mlnL ]
                                totalImpact[metric][layer].append(max(tab1_body_model[model][metric][key][layer][logic]) == max(maxi))
                    
                                tab1_body+= (f'<td>{"<strong>"*int(max(tab1_body_model[model][metric][key][layer][logic]) == max(maxi))}{max(tab1_body_model[model][metric][key][layer][logic])}{"</strong>"*int(max(tab1_body_model[model][metric][key][layer][logic]) == max(maxi))}</td></tr>') if ((y == len(metrics)-1) and (z == len(mlnL)-1 )) else (
                                            f'<td>{"<strong>"*int(max(tab1_body_model[model][metric][key][layer][logic]) == max(maxi))}{max(tab1_body_model[model][metric][key][layer][logic])}{"</strong>"*int(max(tab1_body_model[model][metric][key][layer][logic]) == max(maxi))}</td>')
                            
        tab1_body+= f'<tr> <td colspan="3">Total</td>'
        for y, metric in enumerate(metrics): 
            for z, layer in enumerate(mlnL.keys()):
                maxi = [sum(totalImpact[metric][lay]) for lay in mlnL]
                tab1_body+= (f'<td>{"<strong>"*int(sum(totalImpact[metric][layer]) == max(maxi))}{sum(totalImpact[metric][layer])}{"</strong>"*int(sum(totalImpact[metric][layer]) == max(maxi))}</td></tr>'
                                ) if ((y == len(metrics)-1) and (z == len(mlnL)-1 )) else (
                                f'<td>{"<strong>"*int(sum(totalImpact[metric][layer]) == max(maxi))}{sum(totalImpact[metric][layer])}{"</strong>"*int(sum(totalImpact[metric][layer]) == max(maxi))}</td>')
        
        caption = f'<caption><h2>Legend</h2>{caption_content_lambda(metric)}</caption>'
        table_html = f'<table style="border: 2px solid black; width: 100% !important; background-color: #FFFFFF; color:#000000;">{caption}{tab1_head}{tab1_body}</table>'
        htm = f'<html><head>{style}<title> Best metrics of personalized and global logic for Classic + MLN et Classic + MLN - Att pour chaque modèle et modélisation </title></head><body style="background-color: white;">{table_html}</body></html>'
        
        create_domain(f'{cwd}/analyze_{outputs_path.split("/")[-2]}_made_on_{day}H/{data_folder}/plots/fnTab/tab2')
        timestr = time.strftime("%Y_%m_%d_%H_%M_%S")
        filename1 = f'{cwd}/analyze_{outputs_path.split("/")[-2]}_made_on_{day}H/{data_folder}/plots/fnTab/tab2/Statistical comparaison of approachs in {data_folder} on global logic'+'_'+timestr+'.html'
        _file= open(filename1,"w")
        _file.write(htm)
        _file.close()
        

In [14]:
analyzer_launcher(outputs_name="outputs_16032024", analytical_func=metrics_analyzer_statistics_tab_f_2, layers=layers, approach=approach)

AER
['xgb', 'dtc', 'lrc', 'rfc', 'sv']
['xgb', 'dtc', 'lrc', 'rfc', 'sv']
['xgb', 'dtc', 'lrc', 'rfc', 'sv']
Directory '/Users/djiemboutientcheuvictornico/Documents/M2_Thesis/M2_thesis/scripting/analyze_outputs_16032024_made_on_2024_03_28_09H/AER/plots/fnTab/tab2' created successfully.
AFB
['xgb', 'dtc', 'lrc', 'rfc', 'sv']
['xgb', 'dtc', 'lrc', 'rfc', 'sv']
['xgb', 'dtc', 'lrc', 'rfc', 'sv']
['xgb', 'dtc', 'lrc', 'rfc', 'sv']
['xgb', 'dtc', 'lrc', 'rfc', 'sv']
['xgb', 'dtc', 'lrc', 'rfc', 'sv']
['xgb', 'dtc', 'lrc', 'rfc', 'sv']
['xgb', 'dtc', 'lrc', 'rfc', 'sv']
['xgb', 'dtc', 'lrc', 'rfc', 'sv']
['xgb', 'dtc', 'lrc', 'rfc', 'sv']
['xgb', 'dtc', 'lrc', 'rfc', 'sv']
Directory '/Users/djiemboutientcheuvictornico/Documents/M2_Thesis/M2_thesis/scripting/analyze_outputs_16032024_made_on_2024_03_28_09H/AFB/plots/fnTab/tab2' created successfully.
CREDIT_RISK_DATASET
['xgb', 'dtc', 'lrc', 'rfc', 'sv']
['xgb', 'dtc', 'lrc', 'rfc', 'sv']
['xgb', 'dtc', 'lrc', 'rfc', 'sv']
['xgb', 'dtc', 'lrc',

In [15]:
def metrics_analyzer_statistics_tab_f_3(cols=None, outputs_path=None, cwd=None, data_folder=None, classic_metrics=None, models_name=None, layers=[1], approach=None):
    """ build relevance results about the datasset
    
    Args:
        - cols: list of qualitative variable in the dataset
        - outputs_path: the path where the experimental results are located
        - 
    
    Returns:
        A dedicated folder with those relevante reports and charts
    
    """
    day = time.strftime("%Y_%m_%d_%H")
    if cols != None or classic_metrics != None: # check if cols and classics metrics are filled
        ## analyse of k layer

        # find out all best metric details
        head_lambda = lambda x: f'<tr><td colspan="2" rowspan="2">{x}</td><td colspan="2">Accuracy</td><td colspan="2">Precision</td><td colspan="2">Recall</td><td colspan="2">F1-score</td></tr><tr>{"<td>G</td><td>P</td>"*4}</tr>'
        tab1_head = head_lambda(data_folder)
        tab1_body = ""
        metrics = {
            'accuracy': {
                'classic_mln':{
                    'P':[],
                    'G':[]
                },
                'classic_mln_-_mlna':{
                    'P':[],
                    'G':[]
                }
            },
            'precision': {
                'classic_mln':{
                    'P':[],
                    'G':[]
                },
                'classic_mln_-_mlna':{
                    'P':[],
                    'G':[]
                }
            },
            'recall': {
                'classic_mln':{
                    'P':[],
                    'G':[]
                },
                'classic_mln_-_mlna':{
                    'P':[],
                    'G':[]
                }
            },
            'f1-score': {
                'classic_mln':{
                    'P':[],
                    'G':[]
                },
                'classic_mln_-_mlna':{
                    'P':[],
                    'G':[]
                }
            }
        }

        totalImpact = {
            'accuracy': {
                    'P':[],
                    'G':[]
                },
            'precision': {
                    'P':[],
                    'G':[]
                },
            'recall': {
                    'P':[],
                    'G':[]
                },
            'f1-score': {
                    'P':[],
                    'G':[]
                }
        }
        
        dictio = {
                    'classic_-_mlna': 'Classic - Att',
                    'classic_mln': 'Classic + MLN',
                    'classic_mln_-_mlna': 'Classic + MLN - Att',
                    'classic': 'Classic'
                }
        
        
        style = '<style> table, th, td {border: 1px solid black;border-collapse: collapse;} .wrap-text {word-wrap: break-word;} .wrap-text {overflow-wrap: break-word;} .limited-column {width: 100px;} .dashed-border {border: 1px dashed black;}.dotted-border {border: 1px dotted black;} td {text-align: center;} caption {margin:0; margin-bottom: 2px; text-align: start; border: 1px dashed black;} caption > h2 {text-align: center;}</style>'
        
        caption_content_lambda = lambda x: ''.join([f'<span><strong>{key}</strong>: {value}</span><br>' for key, value in {
            **models_name,
            'MLN k Layer(s)': 'MultiLayer Network with k layer(s)',
            'Att': 'Attributs or modalities of variable(s) used to build MLN',
            'MLN': 'Descriptors extracted from MLN',
            'Classic': f'Learning from classic dataset of {data_folder}',
            'Classic - Att': f'Learning from classic dataset of {data_folder} where Att had been removed',
            'Classic + MLN': f'Learning from classic dataset of {data_folder} where MLN had been added',
            'Classic + MLN - Att': f'Learning from classic dataset of {data_folder} where MLN had been added and Att removed'
            }.items()])
        
        tab1_body_model = {key: deepcopy(metrics) for key in models_name.keys()}

        # fetch layers
        for k in layers:
            # fetch each combinantion of atributs in layers
            for layer_config in get_combinations(range(len(cols)),k): # create subsets of k index of OHE and fetch it
                col_targeted= [f'{cols[i]}' for i in layer_config]
                case_k= '±'.join(col_targeted) if len(layer_config)>1 else col_targeted[0]
                
                ### get files for distincts logic
                match= lambda x: (
                    sum(
                        [
                            re.sub(r'[^\w\s]', '', unidecode(partern)) in re.sub(r'[^\w\s]', '', unidecode(x))
                            for partern in case_k.split("±")
                            ]
                        ) == k if k > 1 else re.sub(r'[^\w\s]', '', unidecode(case_k)) in re.sub(r'[^\w\s]', '', unidecode(x))
                    )
                files = {
                    'global':{
                        'classic': classic_metrics,
                        #'classic_-_mlna':[
                        #    load_data_set_from_url(path=file,sep='\t', encoding='utf-8',index_col=0, na_values=None) 
                        #    for file in get_filenames(
                        #        root_dir=f'{outputs_path}/qualitative/mlna_{k}/data_selection_storage', 
                        #        func=lambda x: ((MLN_C_F(x)) and (match(x))), 
                        #        verbose=False
                         #       )
                         #   ][-1],
                        'classic_mln':[
                            load_data_set_from_url(path=file,sep='\t', encoding='utf-8',index_col=0, na_values=None) 
                            for file in get_filenames(
                                root_dir=f'{outputs_path}/qualitative/mlna_{k}/global/data_selection_storage', 
                                func=lambda x: ((MLN_F(x)) and (match(x))), 
                                verbose=False
                                )
                            ][-1],
                        'classic_mln_-_mlna':[
                            load_data_set_from_url(path=file,sep='\t', encoding='utf-8',index_col=0, na_values=None) 
                            for file in get_filenames(
                                root_dir=f'{outputs_path}/qualitative/mlna_{k}/global/data_selection_storage', 
                                func=lambda x: ((MLN__F(x)) and (match(x))), 
                                verbose=False
                                )
                            ][-1]
                    },
                    "personalized":{
                        'classic': classic_metrics,
                       # 'classic_-_mlna':[
                       #     load_data_set_from_url(path=file,sep='\t', encoding='utf-8',index_col=0, na_values=None) 
                       #     for file in get_filenames(
                       #         root_dir=f'{outputs_path}/qualitative/mlna_{k}/data_selection_storage', 
                       #         func=lambda x: ((MLN_C_F(x)) and (match(x))), 
                       #         verbose=False
                       #         )
                       #     ][-1],
                        'classic_mln':[
                            load_data_set_from_url(path=file,sep='\t', encoding='utf-8',index_col=0, na_values=None) 
                            for file in get_filenames(
                                root_dir=f'{outputs_path}/qualitative/mlna_{k}/personalized/data_selection_storage', 
                                func=lambda x: ((MLN_F(x)) and (match(x))), 
                                verbose=False
                                )
                            ][-1],
                        'classic_mln_-_mlna':[
                            load_data_set_from_url(path=file,sep='\t', encoding='utf-8',index_col=0, na_values=None) 
                            for file in get_filenames(
                                root_dir=f'{outputs_path}/qualitative/mlna_{k}/personalized/data_selection_storage', 
                                func=lambda x: ((MLN__F(x)) and (match(x))), 
                                verbose=False
                                )
                            ][-1]
                    }
                }
                # print(files)
                #outputs[logic] = files
                ### transform and normalize
                models_list = files['personalized']['classic'].index.values.tolist()
                print(models_list)
                metrics = ["accuracy","precision","recall","f1-score"]
                
                # fetch models
                for model in models_list:
                    # fetch evaluation metric
                    for metric in metrics:
                        # fetch approach
                        for i, key in enumerate(approach): 
                            # add metric in the vector
                            tab1_body_model[model][metric][key]['G'].append(round(((round(files['global'][key].loc[model,metric],4) - round(files['global']['classic'].loc[model,metric],4))/round(files['global']['classic'].loc[model,metric],4))*100,4) if round(((round(files['global'][key].loc[model,metric],4) - round(files['global']['classic'].loc[model,metric],4))/round(files['global']['classic'].loc[model,metric],4))*100,4) >= 0 else 0)
                            tab1_body_model[model][metric][key]['P'].append(round(((round(files['personalized'][key].loc[model,metric],4) - round(files['personalized']['classic'].loc[model,metric],4))/round(files['personalized']['classic'].loc[model,metric],4))*100,4) if round(((round(files['personalized'][key].loc[model,metric],4) - round(files['personalized']['classic'].loc[model,metric],4))/round(files['personalized']['classic'].loc[model,metric],4))*100,4) >= 0 else 0)
                            #totalImpact[metric]["P"].append(tab1_body_model[model][metric][key]['P'] >= tab1_body_model[model][metric][key]['G'])
                            #totalImpact[metric]["G"].append(tab1_body_model[model][metric][key]['P'] <= tab1_body_model[model][metric][key]['P'])
        
        # fetch each model
        for model in models_list:
            tab1_body+= f'<tr> <td rowspan="2">{model}</td>'
            # fetch approach
            for i, key in enumerate(files['global'].keys() if approach == None else approach):
                # fetch evaluation metric
                tab1_body+= f'<tr> <td>{dictio[key]}</td>' if i != 0 else f'<td>{dictio[key]}</td>'
                for y, metric in enumerate(metrics):
                    # add metric in the vector
                    totalImpact[metric]["P"].append(max(tab1_body_model[model][metric][key]["G"]) <= max(tab1_body_model[model][metric][key]["P"]))
                    totalImpact[metric]["G"].append(max(tab1_body_model[model][metric][key]["G"]) >= max(tab1_body_model[model][metric][key]["P"]))
        
                    tab1_body+= (f'<td>{"<strong>"*int(max(tab1_body_model[model][metric][key]["G"]) >= max(tab1_body_model[model][metric][key]["P"]))}{max(tab1_body_model[model][metric][key]["G"])}{"</strong>"*int(max(tab1_body_model[model][metric][key]["G"]) >= max(tab1_body_model[model][metric][key]["P"]))}</td>'+
                                f'<td> {"<strong>"*int(max(tab1_body_model[model][metric][key]["G"]) <= max(tab1_body_model[model][metric][key]["P"]))}{max(tab1_body_model[model][metric][key]["P"])}{"</strong>"*int(max(tab1_body_model[model][metric][key]["G"]) <= max(tab1_body_model[model][metric][key]["P"]))}</td>'
                                ) if y != len(metrics)-1 else (f'<td>{"<strong>"*int(max(tab1_body_model[model][metric][key]["G"]) >= max(tab1_body_model[model][metric][key]["P"]))}{max(tab1_body_model[model][metric][key]["G"])}{"</strong>"*int(max(tab1_body_model[model][metric][key]["G"]) >= max(tab1_body_model[model][metric][key]["P"]))}</td>'+
                                f'<td> {"<strong>"*int(max(tab1_body_model[model][metric][key]["G"]) <= max(tab1_body_model[model][metric][key]["P"]))}{max(tab1_body_model[model][metric][key]["P"])}{"</strong>"*int(max(tab1_body_model[model][metric][key]["G"]) <= max(tab1_body_model[model][metric][key]["P"]))}</td></tr>')
        tab1_body+= f'<tr> <td colspan="2">Total</td>'
        for y, metric in enumerate(metrics): 
            tab1_body+= (f'<td>{"<strong>"*int(sum(totalImpact[metric]["G"]) >= sum(totalImpact[metric]["P"]))}{sum(totalImpact[metric]["G"])}{"</strong>"*int(sum(totalImpact[metric]["G"]) >= sum(totalImpact[metric]["P"]))}</td>'+
                            f'<td>{"<strong>"*int(sum(totalImpact[metric]["G"]) <= sum(totalImpact[metric]["P"]))}{sum(totalImpact[metric]["P"])}{"</strong>"*int(sum(totalImpact[metric]["G"]) <= sum(totalImpact[metric]["P"]))}</td>'
                            ) if y != len(metrics)-1 else (f'<td>{"<strong>"*int(sum(totalImpact[metric]["G"]) >= sum(totalImpact[metric]["P"]))}{sum(totalImpact[metric]["G"])}{"</strong>"*int(sum(totalImpact[metric]["G"]) >= sum(totalImpact[metric]["P"]))}</td>'+
                            f'<td>{"<strong>"*int(sum(totalImpact[metric]["G"]) <= sum(totalImpact[metric]["P"]))}{sum(totalImpact[metric]["P"])}{"</strong>"*int(sum(totalImpact[metric]["G"]) <= sum(totalImpact[metric]["P"]))}</td></tr>')
            
        
        caption = f'<caption><h2>Legend</h2>{caption_content_lambda(metric)}</caption>'
        table_html = f'<table style="border: 2px solid black; width: 100% !important; background-color: #FFFFFF; color:#000000;">{caption}{tab1_head}{tab1_body}</table>'
        htm = f'<html><head>{style}<title> Best outperforming metrics of personalized and global logic for Classic + MLN et Classic + MLN - Att pour chaque modèle </title></head><body style="background-color: white;">{table_html}</body></html>'
        
        create_domain(f'{cwd}/analyze_{outputs_path.split("/")[-2]}_made_on_{day}H/{data_folder}/plots/fnTab/tab3')
        timestr = time.strftime("%Y_%m_%d_%H_%M_%S")
        filename1 = f'{cwd}/analyze_{outputs_path.split("/")[-2]}_made_on_{day}H/{data_folder}/plots/fnTab/tab3/Statistical comparaison of approachs in {data_folder} on global logic'+'_'+timestr+'.html'
        _file= open(filename1,"w")
        _file.write(htm)
        _file.close()
        

In [ ]:
analyzer_launcher(outputs_name="outputs_16032024", analytical_func=metrics_analyzer_statistics_tab_f_3, layers=layers, approach=approach)

In [17]:
def metrics_analyzer_statistics_tab_f_4(cols=None, outputs_path=None, cwd=None, data_folder=None, classic_metrics=None, models_name=None, layers=[1], approach=None):
    """ build relevance results about the datasset
    
    Args:
        - cols: list of qualitative variable in the dataset
        - outputs_path: the path where the experimental results are located
        - 
    
    Returns:
        A dedicated folder with those relevante reports and charts
    
    """
    day = time.strftime("%Y_%m_%d_%H")
    if cols != None or classic_metrics != None: # check if cols and classics metrics are filled
        ## analyse of k layer

        # find out all best metric details
        mlnL = {f'MLN {key}': {
                    'P':[],
                    'G':[]
                } for key in layers}
        totalImpact = {
            'accuracy': {f'MLN {key}': [] for key in layers},
            'precision': {f'MLN {key}': [] for key in layers},
            'recall': {f'MLN {key}': [] for key in layers},
            'f1-score': {f'MLN {key}': [] for key in layers}
        }
        head_lambda = lambda x: f'<tr><td colspan="3" rowspan="2">{x}</td><td colspan="{len(mlnL)}">Accuracy</td><td colspan="{len(mlnL)}">Precision</td><td colspan="{len(mlnL)}">Recall</td><td colspan="{len(mlnL)}">F1-score</td></tr><tr>{"".join(["<td>"+key+"</td>" for key in mlnL.keys()]) *4}</tr>'
        tab1_head = head_lambda(data_folder)
        tab1_body = ""
        
        metrics = {
            'accuracy': {
                'classic_mln':deepcopy(mlnL),
                'classic_mln_-_mlna':deepcopy(mlnL)
            },
            'precision': {
                'classic_mln':deepcopy(mlnL),
                'classic_mln_-_mlna':deepcopy(mlnL)
            },
            'recall': {
                'classic_mln':deepcopy(mlnL),
                'classic_mln_-_mlna':deepcopy(mlnL)
            },
            'f1-score': {
                'classic_mln':deepcopy(mlnL),
                'classic_mln_-_mlna':deepcopy(mlnL)
            }
        }
        
            
        dictio = {
            'classic_-_mlna': 'Classic - Att',
            'classic_mln': 'Classic + MLN',
            'classic_mln_-_mlna': 'Classic + MLN - Att',
            'classic': 'Classic'
        }
        
        style = '<style> table, th, td {border: 1px solid black;border-collapse: collapse;} .wrap-text {word-wrap: break-word;} .wrap-text {overflow-wrap: break-word;} .limited-column {width: 100px;} .dashed-border {border: 1px dashed black;}.dotted-border {border: 1px dotted black;} td {text-align: center;} caption {margin:0; margin-bottom: 2px; text-align: start; border: 1px dashed black;} caption > h2 {text-align: center;}</style>'
        
        caption_content_lambda = lambda x: ''.join([f'<span><strong>{key}</strong>: {value}</span><br>' for key, value in {
            **models_name,
            'MLN k Layer(s)': 'MultiLayer Network with k layer(s)',
            'Att': 'Attributs or modalities of variable(s) used to build MLN',
            'MLN': 'Descriptors extracted from MLN',
            'Classic': f'Learning from classic dataset of {data_folder}',
            'Classic - Att': f'Learning from classic dataset of {data_folder} where Att had been removed',
            'Classic + MLN': f'Learning from classic dataset of {data_folder} where MLN had been added',
            'Classic + MLN - Att': f'Learning from classic dataset of {data_folder} where MLN had been added and Att removed'
            }.items()])
        
        tab1_body_model = {key: deepcopy(metrics) for key in models_name.keys()}

        # fetch layers
        for k in layers:
            # fetch each combinantion of atributs in layers
            for layer_config in get_combinations(range(len(cols)),k): # create subsets of k index of OHE and fetch it
                col_targeted= [f'{cols[i]}' for i in layer_config]
                case_k= '±'.join(col_targeted) if len(layer_config)>1 else col_targeted[0]
                
                ### get files for distincts logic
                match= lambda x: (
                    sum(
                        [
                            re.sub(r'[^\w\s]', '', unidecode(partern)) in re.sub(r'[^\w\s]', '', unidecode(x))
                            for partern in case_k.split("±")
                            ]
                        ) == k if k > 1 else re.sub(r'[^\w\s]', '', unidecode(case_k)) in re.sub(r'[^\w\s]', '', unidecode(x))
                    )
                files = {
                    'global':{
                        'classic': classic_metrics,
                        #'classic_-_mlna':[
                        #    load_data_set_from_url(path=file,sep='\t', encoding='utf-8',index_col=0, na_values=None) 
                        #    for file in get_filenames(
                        #        root_dir=f'{outputs_path}/qualitative/mlna_{k}/data_selection_storage', 
                        #        func=lambda x: ((MLN_C_F(x)) and (match(x))), 
                        #        verbose=False
                         #       )
                         #   ][-1],
                        'classic_mln':[
                            load_data_set_from_url(path=file,sep='\t', encoding='utf-8',index_col=0, na_values=None) 
                            for file in get_filenames(
                                root_dir=f'{outputs_path}/qualitative/mlna_{k}/global/data_selection_storage', 
                                func=lambda x: ((MLN_F(x)) and (match(x))), 
                                verbose=False
                                )
                            ][-1],
                        'classic_mln_-_mlna':[
                            load_data_set_from_url(path=file,sep='\t', encoding='utf-8',index_col=0, na_values=None) 
                            for file in get_filenames(
                                root_dir=f'{outputs_path}/qualitative/mlna_{k}/global/data_selection_storage', 
                                func=lambda x: ((MLN__F(x)) and (match(x))), 
                                verbose=False
                                )
                            ][-1]
                    },
                    "personalized":{
                        'classic': classic_metrics,
                       # 'classic_-_mlna':[
                       #     load_data_set_from_url(path=file,sep='\t', encoding='utf-8',index_col=0, na_values=None) 
                       #     for file in get_filenames(
                       #         root_dir=f'{outputs_path}/qualitative/mlna_{k}/data_selection_storage', 
                       #         func=lambda x: ((MLN_C_F(x)) and (match(x))), 
                       #         verbose=False
                       #         )
                       #     ][-1],
                        'classic_mln':[
                            load_data_set_from_url(path=file,sep='\t', encoding='utf-8',index_col=0, na_values=None) 
                            for file in get_filenames(
                                root_dir=f'{outputs_path}/qualitative/mlna_{k}/personalized/data_selection_storage', 
                                func=lambda x: ((MLN_F(x)) and (match(x))), 
                                verbose=False
                                )
                            ][-1],
                        'classic_mln_-_mlna':[
                            load_data_set_from_url(path=file,sep='\t', encoding='utf-8',index_col=0, na_values=None) 
                            for file in get_filenames(
                                root_dir=f'{outputs_path}/qualitative/mlna_{k}/personalized/data_selection_storage', 
                                func=lambda x: ((MLN__F(x)) and (match(x))), 
                                verbose=False
                                )
                            ][-1]
                    }
                }
                # print(files)
                #outputs[logic] = files
                ### transform and normalize
                models_list = files['personalized']['classic'].index.values.tolist()
                print(models_list)
                metrics = ["accuracy","precision","recall","f1-score"]
                
                # fetch models
                for model in models_list:
                    # fetch evaluation metric
                    for metric in metrics:
                        # fetch approach
                        for i, key in enumerate(approach): 
                            # add metric in the vector
                            tab1_body_model[model][metric][key][f'MLN {k}']["G"].append(round(((round(files['global'][key].loc[model,metric],4) - round(files['global']['classic'].loc[model,metric],4))/round(files['global']['classic'].loc[model,metric],4))*100,4) if round(((round(files['global'][key].loc[model,metric],4) - round(files['global']['classic'].loc[model,metric],4))/round(files['global']['classic'].loc[model,metric],4))*100,4) >= 0 else 0)
                            tab1_body_model[model][metric][key][f'MLN {k}']["P"].append(round(((round(files['personalized'][key].loc[model,metric],4) - round(files['personalized']['classic'].loc[model,metric],4))/round(files['personalized']['classic'].loc[model,metric],4))*100,4) if round(((round(files['personalized'][key].loc[model,metric],4) - round(files['personalized']['classic'].loc[model,metric],4))/round(files['personalized']['classic'].loc[model,metric],4))*100,4) >= 0 else 0)
                            
        logics = {"G":"Global","P":"Personalized"}    
        # fetch each model
        for model in models_list:
                tab1_body+= f'<tr> <td rowspan="4">{model}</td>'
                # fetch approach
                for i, key in enumerate(files['global'].keys() if approach == None else approach):
                    # fetch evaluation metric
                    tab1_body+= (f'<tr> <td rowspan="2">{dictio[key]}</td>') if i != 0 else (f'<td rowspan="2">{dictio[key]}</td>')
                    for l,logic in enumerate(logics.keys()):
                        tab1_body+= (f'<tr><td>{logics[logic]}</td>') if l != 0 else (f'<td>{logics[logic]}</td>')
                        for y, metric in enumerate(metrics):
                            #print(f"{y}--")
                            # fetch layers
                            for z, layer in enumerate(mlnL.keys()):
                                #print(f"{z};")
                                # add metric in the vector
                                maxi = [max(tab1_body_model[model][metric][key][lay][logic]) for lay in mlnL ]
                                totalImpact[metric][layer].append(max(tab1_body_model[model][metric][key][layer][logic]) == max(maxi))
                    
                                tab1_body+= (f'<td>{"<strong>"*int(max(tab1_body_model[model][metric][key][layer][logic]) == max(maxi))}{max(tab1_body_model[model][metric][key][layer][logic])}{"</strong>"*int(max(tab1_body_model[model][metric][key][layer][logic]) == max(maxi))}</td></tr>') if ((y == len(metrics)-1) and (z == len(mlnL)-1 )) else (
                                            f'<td>{"<strong>"*int(max(tab1_body_model[model][metric][key][layer][logic]) == max(maxi))}{max(tab1_body_model[model][metric][key][layer][logic])}{"</strong>"*int(max(tab1_body_model[model][metric][key][layer][logic]) == max(maxi))}</td>')
                            
        tab1_body+= f'<tr> <td colspan="3">Total</td>'
        for y, metric in enumerate(metrics): 
            for z, layer in enumerate(mlnL.keys()):
                maxi = [sum(totalImpact[metric][lay]) for lay in mlnL]
                tab1_body+= (f'<td>{"<strong>"*int(sum(totalImpact[metric][layer]) == max(maxi))}{sum(totalImpact[metric][layer])}{"</strong>"*int(sum(totalImpact[metric][layer]) == max(maxi))}</td></tr>'
                                ) if ((y == len(metrics)-1) and (z == len(mlnL)-1 )) else (
                                f'<td>{"<strong>"*int(sum(totalImpact[metric][layer]) == max(maxi))}{sum(totalImpact[metric][layer])}{"</strong>"*int(sum(totalImpact[metric][layer]) == max(maxi))}</td>')
        
        caption = f'<caption><h2>Legend</h2>{caption_content_lambda(metric)}</caption>'
        table_html = f'<table style="border: 2px solid black; width: 100% !important; background-color: #FFFFFF; color:#000000;">{caption}{tab1_head}{tab1_body}</table>'
        htm = f'<html><head>{style}<title> Best outperforming metrics of personalized and global logic for Classic + MLN et Classic + MLN - Att pour chaque modèle et modélisation </title></head><body style="background-color: white;">{table_html}</body></html>'
        
        create_domain(f'{cwd}/analyze_{outputs_path.split("/")[-2]}_made_on_{day}H/{data_folder}/plots/fnTab/tab4')
        timestr = time.strftime("%Y_%m_%d_%H_%M_%S")
        filename1 = f'{cwd}/analyze_{outputs_path.split("/")[-2]}_made_on_{day}H/{data_folder}/plots/fnTab/tab4/Statistical comparaison of approachs in {data_folder} on global logic'+'_'+timestr+'.html'
        _file= open(filename1,"w")
        _file.write(htm)
        _file.close()
        

In [ ]:
analyzer_launcher(outputs_name="outputs_16032024", analytical_func=metrics_analyzer_statistics_tab_f_4, layers=layers, approach=approach)